In [3]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [11]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

corpus length: 600901
total chars: 59
nb sequences: 200287
Vectorization...


In [25]:
y.shape

(200287, 59)

In [22]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [23]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [24]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
200287/200287 [==============================] - 148s - loss: 1.9893   

----- diversity: 0.2
----- Generating with seed: " europe there is now perhaps
no one in t"
 europe there is now perhaps
no one in the conduptive in the pression of the sould and the contrany of the conduptive with the more there is the man there in the complest there is the conduct of the more there is the realing of the realing of the religion of the sould in the rearents of the conception of the self conduptive of the condince of the contrances of the realing of the something of the exploson the contrany there is the condin

----- diversity: 0.5
----- Generating with seed: " europe there is now perhaps
no one in t"
 europe there is now perhaps
no one in the whom in the ingerming, there it has their mander of
all the presost lite the man has simple and all over instilist in the more could them light in the one belong there in the mist and 

which our grandfathers used to hang be new gerald to he
have i euac
bless exacct all sessiles has even the ensuation. the     langes for he vaniss! he est
mery man."--ar a him broke super-its to strengtherlatifus philosophers to passivelings,--is to exceptive of
self-conceious been: "these not at world heart--to that complion, this curcism arrive, not
doc above forced thry can dun whomy of
more sociatur, himself, it syet is longed, tr

----- diversity: 1.2
----- Generating with seed: ",
which our grandfathers used to hang be"
,
which our grandfathers used to hang becays,
but whum it dogscultagles,"
--when trappy a woulding
that now
themshing. that that dotronn
of the life "ous freeknow. the relay.ly feveed 
quevifal
fadules haw dicounbee imsuvren frenry of nearn for, geneanor. sufk of own father,ces it coutpope a perhaps nor know the univer countedly spies againallung!
that "the wouldex and soulesby, that his trivol-deringw""igo: pravity to menbalized.

1ea 

--------------------------

the spectacle of the tartuffery of old knowledge of the present and always the subsided the seems to a morality of the seems to the seems and consequently the moral present and and the and evil and and strict of the morality. the subsided the subsided the instinction of the subsided the strict and and the end that the morality of the ascribe the subsided the moral seems to the interpretated to the means of the substance of the the work 

----- diversity: 0.5
----- Generating with seed: "the spectacle of the tartuffery of old k"
the spectacle of the tartuffery of old knowledge, the work and action with he will to say, with the have the extern is not conditional of ourselves a distance--and reem, one should all think and who accordingly everything of the morals. the power that consequetute of the sort of the subtletions of love in places that a the but be admoc of all and consequently constance, as the so inspiring it is the most interpretation of this
everythin

----- diversity: 1.0
---

the thing-belaps? on rudeps--a: a first conlect for sucheps astroworm of an rese indieity of lide correge philosophy, this skeptere
hands to be
this iter and cerairce, everychind "belongs the parttingifice opression, an impulses self will should
abolths of being.


4

=things. do their 

----- diversity: 1.2
----- Generating with seed: "is administered it proves a potent stimu"
is administered it proves a potent stimuuation ef, i wince along bealified
ynowing, so
proted have alto individual natuined this
wasces. it at lant" in i way
unimage also. chraspiceme
ley and action it is now a marking opaith y "do no hivt implificu to parative, each of?, its learning ideas philosofhe, the commanlying clavely; he nowerwevactions, and glority
which seexiess is, such belitureding dangect form, first is not go looks distro

--------------------------------------------------
Iteration 12
Epoch 1/1
200287/200287 [==============================] - 154s - loss: 1.3812   

----- diversity: 0.2
----- Gen

has been surmounted? schumann, fleeing in the problems of the subject of the more to the same to the conceited the subject of the frenchmsed the sensation of the superficial and subject of the conscience and and the sensible of the contemply the consider to the considered to the supernature the person of the suffering the probably the subject of the subject of the consider to the suffering the subject of the suffering the sensation of t

----- diversity: 0.5
----- Generating with seed: "has been surmounted? schumann, fleeing i"
has been surmounted? schumann, fleeing it has not the reason and the sensation of the weak of the neiven the dissemition for the belief in the very world, the
other way granters and hand, or to a the free of the forey which everything the the sensation of form of the finally as the habits, one is to such and can that of which the entibly to the purposes to the good moralist to the reference to be the happility of the same of some to say

----- diversity: 1.0
---

...

more stands and the states, and the world to the same as the more consequently to a strange, and the saint to the any believes that is the man with the power of the same as the worth, and in the state of the case of the same to the same as the will to the world to the same as the world to the man is the state of the state of the master and states and contemption of the personal and the proper to th

----- diversity: 0.5
----- Generating with seed: "is point of view there is perhaps much
m"
is point of view there is perhaps much
more for their contain itself as a feeling of the form of stands as the case of all the formen of such master there is a common one stands the proceeding, and also are power of every personal in the same of the happiness and prompting and all power of the world to the foregrous and with the propensible with their contemples the development of an religious philosophical the intercourses and the
subs

----- diversity: 1.0
----- Generating with seed: "is point of v

of his rowd--if wely of the prejustures lakes itself discoval case, disciplinally behinness"
aes, and evil lendous and fits to le
to awaptire its love concernin

----- diversity: 1.2
----- Generating with seed: "lsion of this concession, one has the sa"
lsion of this concession, one has the salvarity which it becomes and forning bolt," they aro, it is
a wifach, that it is unfortlece because "good of bedod a"fertencess not whatest clear-did bagy and
spirit bee. what abidul more-rimencicanitys
giving can bewrack only
privilegy, thesolink than "tohos that it may finally."--is no feward on
it" avainime goes no means
of "sants and noced. with an ancondemaally reventive resiteop: oppasiona u

--------------------------------------------------
Iteration 47
Epoch 1/1
200287/200287 [==============================] - 147s - loss: 1.3060   

----- diversity: 0.2
----- Generating with seed: "n the
voice, the expression, the bearing"
n the
voice, the expression, the bearing the strength of the pres

ore fundamental value for life generally and all the superiory and super-idmen of the subject in the subfirie in the many and something in the same of the other hand, and and the substand and in the same antithes! the subject in the substand and in the same and has all the subserve and in the substance and in the subject of the substance of the superior and something is a present and the subject in the substance and in the streems and a

----- diversity: 0.5
----- Generating with seed: "ore fundamental value for life generally"
ore fundamental value for life generally and look and such a religious and super-love the consequently say, in reality in the passion all logic for an instinct, the like the ancient and which we are commanding and original and subject grown the more itself which is the contradiction of the antithesis and so are the fact that the probably are the more so the soul, the and the asception of the sublime and when could the too "and that to a

----- diversity: 1.0
---

more than it is experiencely thought--it is not be the preservitude--who ligt
at!r
who individual every this is, if, the mataled are the feeling to means against and distress as d

----- diversity: 1.2
----- Generating with seed: "at separate man from
man:--such men, wit"
at separate man from
man:--such men, with peoniy asconed to faith calling suffering. ay or -law demountry
must its. he free.ver
atters ascerent apprexicai forces litine in relationanism tortorishy." in such calctorian callectelity
i distrul "necessity so
solitude--preferect norsed sacrplific scorn and immevials its sufferir whether vomant, that, subselus!
hearest is onebant if "freedom,
else apprioatic agae the artific dispude, consider

--------------------------------------------------
Iteration 54
Epoch 1/1
200287/200287 [==============================] - 149s - loss: 1.3055   

----- diversity: 0.2
----- Generating with seed: "his "earth-residuum," and perhaps some d"
his "earth-residuum," and perhaps some distinc

adore; indeed, every instance of taking and in the soul of the same and the soul of the most something of the soul of the ancient of the most lives the same as the present the same satisfaction of the super-any in the soul of the same and to the soul of the same and the same as the saint of the super-all the depths in the same as the superstion of the entire the same and the superstion of the soul of the soul of the interman and in the 

----- diversity: 0.5
----- Generating with seed: "adore; indeed, every instance of taking "
adore; indeed, every instance of taking and to danger and and to see the sight of morals,
in the presurved the dangerous and in the most and the most strange to make the most excitation of the instinct of things of compassify of the super-id! the evil
of the soul of the morals and excitation of all every inconsidenced the exception and that is the interrorally and all the soul of the world-promils of think the consequences. the other or

----- diversity: 1.0
---